In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors
import matplotlib.ticker as ptick
from matplotlib.ticker import FixedLocator
import matplotlib.dates as mdates
from SharedX import ShareXaxis
from UniversalColor import UniversalColor
from legend_shadow import legend_shadow
from scipy.io import readsav
import datetime

import JupiterMag as jm
import Leadangle_wave as LeadA

from Leadangle_fit_JunoUVS import Obsresults
from Leadangle_fit_JunoUVS import viewingangle
from Leadangle_fit_JunoUVS import calc_eqlead
from Leadangle_fit_JunoUVS import local_time_moon

import spiceypy as spice
spice.furnsh('kernel/cassMetaK.txt')

UC = UniversalColor()
UC.set_palette()

F = ShareXaxis()
F.fontsize = 23
F.fontname = 'Liberation Sans Narrow'
F.set_default()

Importing Library
done


In [2]:
# Input about Juno observation
TARGET_MOON = 'Europa'
TARGET_FP = ['MAW', 'TEB']
TARGET_HEM = 'both'
PJ_LIST = [62]
PJ_LIST = [1, 3]+np.arange(4, 68+1, 1).tolist()

In [3]:
# %% Constants
MU0 = 1.26E-6            # 真空中の透磁率
AMU2KG = 1.66E-27        # 原子質量をkgに変換するファクタ [kg]
RJ = 71492E+3            # JUPITER RADIUS [m]
MJ = 1.90E+27            # JUPITER MASS [kg]
C = 2.99792E+8           # LIGHT SPEED [m/s]
G = 6.67E-11             # 万有引力定数  [m^3 kg^-1 s^-2]

Psyn_io = (12.89)*3600      # Moon's synodic period [sec]
Psyn_eu = (11.22)*3600      # Moon's synodic period [sec]
Psyn_ga = (10.53)*3600      # Moon's synodic period [sec]

In [4]:
# Select moon synodic orbital period
if TARGET_MOON == 'Io':
    Psyn = Psyn_io
    r_moon = 5.9*RJ
elif TARGET_MOON == 'Europa':
    Psyn = Psyn_eu
    r_moon = 9.4*RJ
elif TARGET_MOON == 'Ganymede':
    Psyn = Psyn_ga
    r_moon = 15.0*RJ

In [5]:
# Select moon synodic orbital period
if len(PJ_LIST) > 1:
    if TARGET_MOON == 'Io':
        Psyn = Psyn_io
        PJ_LIST.pop(54-2)
        PJ_LIST.pop(55-3)
        PJ_LIST.pop(56-4)
        PJ_LIST.pop(57-5)
        PJ_LIST.pop(61-6)
        PJ_LIST.pop(63-7)
        PJ_LIST.pop(64-8)
        PJ_LIST.pop(65-9)
        PJ_LIST.pop(67-10)
        r_moon = 5.9*RJ
    elif TARGET_MOON == 'Europa':
        Psyn = Psyn_eu
        PJ_LIST.pop(24-2)
        PJ_LIST.pop(43-3)
        PJ_LIST.pop(47-4)
        PJ_LIST.pop(49-5)
        PJ_LIST.pop(50-6)
        PJ_LIST.pop(51-7)
        PJ_LIST.pop(53-8)
        PJ_LIST.pop(55-9)
        PJ_LIST.pop(56-10)
        PJ_LIST.pop(60-11)
        PJ_LIST.pop(61-12)
        PJ_LIST.pop(63-13)
        PJ_LIST.pop(64-14)
        PJ_LIST.pop(65-15)
        PJ_LIST.pop(66-16)
        PJ_LIST.pop(67-17)
        PJ_LIST.pop(68-18)
        r_moon = 9.4*RJ
    elif TARGET_MOON == 'Ganymede':
        Psyn = Psyn_ga
        PJ_LIST.pop(24-2)
        PJ_LIST.pop(31-3)
        PJ_LIST.pop(39-4)
        PJ_LIST.pop(43-5)
        PJ_LIST.pop(44-6)
        PJ_LIST.pop(45-7)
        PJ_LIST.pop(51-8)
        PJ_LIST.pop(52-9)
        PJ_LIST.pop(53-10)
        PJ_LIST.pop(54-11)
        PJ_LIST.pop(55-12)
        PJ_LIST.pop(56-13)
        PJ_LIST.pop(61-14)
        PJ_LIST.pop(62-15)
        PJ_LIST.pop(63-16)
        PJ_LIST.pop(64-17)
        PJ_LIST.pop(65-18)
        PJ_LIST.pop(66-19)
        PJ_LIST.pop(67-20)
        PJ_LIST.pop(68-21)
        r_moon = 15.0*RJ

    print(PJ_LIST)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 48, 52, 54, 57, 58, 59, 62]


In [6]:
# Data
wlon_fp, err_wlon_fp, lat_fp, err_lat_fp, moon_S3wlon, et_fp, hem_fp, pj_fp = Obsresults(
    PJ_LIST, TARGET_MOON, TARGET_FP, TARGET_HEM=TARGET_HEM, FLIP=False
)

eqlead_fp, eqlead_fp_0, eqlead_fp_1, wlon_fp_eq = calc_eqlead(wlon_fp,
                                                              err_wlon_fp,
                                                              lat_fp,
                                                              err_lat_fp,
                                                              hem_fp,
                                                              moon_S3wlon,
                                                              TARGET_MOON)

# Emission angle
view_angle = np.zeros(3)
for i in PJ_LIST:
    for j in TARGET_FP:
        view_angle1 = viewingangle(PJnum=i,
                                   target_moon=TARGET_MOON,
                                   target_fp=j,
                                   target_hem=TARGET_HEM,
                                   FLIP=False)
        view_angle = np.append(view_angle, view_angle1)

# 余計な部分を削除
view_angle = view_angle[3:]

# Emission angleで制限
view_idx = np.where((view_angle <= 30.0))

wlon_fp = wlon_fp[view_idx]
err_wlon_fp = err_wlon_fp[view_idx]
lat_fp = lat_fp[view_idx]
err_lat_fp = err_lat_fp[view_idx]
moon_S3wlon = moon_S3wlon[view_idx]
et_fp = et_fp[view_idx]
hem_fp = hem_fp[view_idx]
pj_fp = pj_fp[view_idx]
view_angle = view_angle[view_idx]

print(wlon_fp.shape)

(343,)


In [7]:
def calc_r_surf(r0, r1, lat):
    """
    `lat` ... latitude [rad]
    """
    rs = r0*r1/np.sqrt((r0*np.sin(lat))**2+(r1*np.cos(lat))**2)  # [m]

    return rs

In [8]:
# 経度0度(y=0)平面のx-z対応テーブル (900km高度)
extradius = np.loadtxt('data/Alt_900km/rthetaphi.txt')
r_e = extradius[0, :]        # [RJ]
theta_e = np.radians(extradius[1, :])    # [rad]
phi_e = np.radians(extradius[2, :])      # [rad]

In [ ]:
# 中央値
rho_arr = np.zeros(wlon_fp.size)
z_arr = np.zeros(wlon_fp.size)
mu_i_arr = np.zeros(wlon_fp.size)

# 磁場モデルの設定
mu_i_default = 139.6    # default: 139.6 [nT]
mu_i_coeff = np.arange(0, 2.1, 0.05)
jm.Internal.Config(Model="jrm33", CartesianIn=True,
                   CartesianOut=True, Degree=18)

r0 = 1.0*RJ                 # [m]
r1 = 1.0*RJ*(14.4/15.4)     # [m]

for i in range(rho_arr.size):
    latitude = lat_fp[i]
    theta = np.radians(90.0-latitude)
    phi = np.radians(360.0-wlon_fp[i])

    # テーブルを参照し距離を確定
    dis = np.abs(theta-theta_e)
    idx = np.argmin(dis)
    r = r_e[idx]

    x0 = r*np.sin(theta)*np.cos(phi)
    y0 = r*np.sin(theta)*np.sin(phi)
    z0 = r*np.cos(theta)

    rho_j_arr = np.zeros(mu_i_coeff.size)
    for j in range(mu_i_coeff.size):
        jm.Con2020.Config(mu_i=mu_i_default*mu_i_coeff[j],
                          equation_type='analytic')
        # create trace objects, pass starting position(s) x0,y0,z0
        T1 = jm.TraceField(x0, y0, z0,
                           IntModel='jrm33', ExtModel='Con2020',
                           MaxStep=0.0003,
                           MaxLen=800000, ErrMax=0.000001)

        x1 = T1.x[0][~np.isnan(T1.x[0])]
        y1 = T1.y[0][~np.isnan(T1.y[0])]
        z1 = T1.z[0][~np.isnan(T1.z[0])]
        rho = np.sqrt(x1**2 + y1**2 + z1**2)

        # Satellite orbital plane
        idx_z0 = np.argmin(np.abs(z1))
        rho_j_arr[j] = rho[idx_z0]

    j_index = np.argmin(np.abs(rho_j_arr-r_moon/RJ))

    rho_arr[i] = rho_j_arr[j_index]
    z_arr = T1.z[0][idx_z0]
    mu_i_arr[i] = mu_i_default * mu_i_coeff[j_index]